### Importar y Cargar Datos

In [1]:
import numpy as np
import pandas as pd
import os

# ---------------------------
# CARGA DE ARCHIVO
# ---------------------------
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/FPC1C2.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")
df.head()

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx


,Unnamed: 0,SERIE,FECHA,C1 H1 %,C1 H2 %,C1 H3 %,C1 H0 %,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF,C2 H1 %,C2 H2 %,C2 H3 %,C2 H0 %,C2 H1 pF,C2 H2 pF,C2 H3 pF,C2 H0 pF
0,NaN,59571,2016-10-23,0.34,NaN,NaN,0.37,280.25,NaN,NaN,190.91,0.21,NaN,NaN,6.33,1114.5,NaN,NaN,373.17
1,NaN,59571,2006-01-29,0.36,NaN,NaN,0.40,281.20,NaN,NaN,191.80,0.19,NaN,NaN,0.25,1053.0,NaN,NaN,360.60
2,NaN,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,59572,2006-01-29,0.37,NaN,NaN,0.34,288.10,NaN,NaN,192.50,0.21,NaN,NaN,0.26,1072.0,NaN,NaN,353.60


### Limpieza de Datos

In [2]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------

df = df.drop(columns=df.columns[0])  # elimina la primera columna vacía
cols_base = ['SERIE', 'FECHA']
cols_fp = [col for col in df.columns if col.endswith('%')]

df_fp = df[cols_base + cols_fp]  # dataset base solo con %s
df_fp.head()

,SERIE,FECHA,C1 H1 %,C1 H2 %,C1 H3 %,C1 H0 %,C2 H1 %,C2 H2 %,C2 H3 %,C2 H0 %
0,59571,2016-10-23,0.34,NaN,NaN,0.37,0.21,NaN,NaN,6.33
1,59571,2006-01-29,0.36,NaN,NaN,0.40,0.19,NaN,NaN,0.25
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,0.37,NaN,NaN,0.34,0.21,NaN,NaN,0.26


### SubTabla C1

In [3]:
# ---------------------------
# SUBTABLA C1
# ---------------------------
cols_fp_c1 = [col for col in df_fp.columns if col.startswith('C1')]
df_fp_c1 = df_fp[cols_base + cols_fp_c1].copy()

# máximo de FP C1 (solo auxiliar, no se mostrará en tablas finales)
df_fp_c1['Max_FP_C1'] = df_fp_c1[cols_fp_c1].max(axis=1)

df_fp_c1.head()

,SERIE,FECHA,C1 H1 %,C1 H2 %,C1 H3 %,C1 H0 %,Max_FP_C1
0,59571,2016-10-23,0.34,NaN,NaN,0.37,0.37
1,59571,2006-01-29,0.36,NaN,NaN,0.40,0.40
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,0.37,NaN,NaN,0.34,0.37


### Configuración del Puntaje

In [4]:
# ---------------------------
# CONFIGURACIÓN DE PUNTAJE
# ---------------------------
pesos = [5, 3, 1]  
limites = {"FP": [1.0, 0.5]}  # [límite superior, límite inferior]

def asignar_puntaje(df, columna_valor, columna_puntaje, limites, pesos):
    df[columna_puntaje] = np.select(
        [
            df[columna_valor] > limites[0],
            (df[columna_valor] > limites[1]) & (df[columna_valor] <= limites[0]),
            df[columna_valor] <= limites[1]
        ],
        pesos,
        default=np.nan
    )
    return df

### Asignación de puntajes

In [5]:
# aplicar regla IEEE C57.152-2013
df_fp_c1 = asignar_puntaje(df_fp_c1, 'Max_FP_C1', 'FPBC1', limites["FP"], pesos)

# ---------------------------
# TABLAS BASE
# ---------------------------
df_FP_C1 = df_fp_c1[['SERIE', 'FECHA', 'FPBC1']].copy()   # tabla base

# tabla detallada → quitamos Max_FP_C1 y movemos FPBC1 a la tercera columna
df_full_C1 = df_fp_c1[['SERIE', 'FECHA', 'FPBC1'] + cols_fp_c1].copy()

### Extensión de calenadrio desde el 2015 al dia de hoy

In [6]:
# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2015
# ---------------------------
inicio = "2015-01-01"
fecha_inicio = pd.Timestamp(inicio)
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- tabla extendida FPBC1 ----------
ultimos_2015 = df_FP_C1[df_FP_C1['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2015['FECHA'] = fecha_inicio
base_ext = pd.concat([df_FP_C1, ultimos_2015], ignore_index=True)

df_extendida_FP_C1 = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida_FP_C1 = df_extendida_FP_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- tabla detallada extendida ----------
ultimos_2015_det = df_full_C1[df_full_C1['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2015_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full_C1, ultimos_2015_det], ignore_index=True)

df_detalles_ext_FP_C1 = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext_FP_C1 = df_detalles_ext_FP_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_39324\533060686.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_FP_C1 = df_extendida_FP_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_39324\533060686.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_detalles_ext_FP_C1 = df_detalles_ext_FP_C1.groupby("SERIE").apply(lambda g: 

### Funciones estándares a llamar

In [7]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_FP_C1():
    return df_FP_C1

def get_df_extendida_FP_C1():
    return df_extendida_FP_C1

def get_df_detalles_FP_C1():
    return df_full_C1

def get_df_detalles_ext_FP_C1():
    return df_detalles_ext_FP_C1

# ---------------------------
# DEMO DE RESULTADOS
# ---------------------------
print("\n ====== TABLA FPBC1 ORIGINAL ====== \n")
print(get_df_FP_C1().tail(), "\n")

print("\n ====== TABLA FPBC1 EXTENDIDA ====== \n")
print(get_df_extendida_FP_C1().tail(), "\n")

print("\n ====== TABLA DETALLADA FPBC1 ====== \n")
print(get_df_detalles_FP_C1().head(), "\n")

print("\n ====== TABLA DETALLADA EXTENDIDA FPBC1 ====== \n")
print(get_df_detalles_ext_FP_C1().tail(), "\n")




 ====== TABLA FPBC1 ORIGINAL ====== 

         SERIE      FECHA  FPBC1
226  PT9003-01 2003-02-26    NaN
227  PT9003-01 2000-04-12    NaN
228    V101038 2024-04-13    1.0
229    V101038 2017-11-02    1.0
230    V101041 2017-10-17    1.0 


 ====== TABLA FPBC1 EXTENDIDA ====== 

          SERIE      FECHA  FPBC1
336598  V101041 2025-10-30    1.0
336599  V101041 2025-10-31    1.0
336600  V101041 2025-11-01    1.0
336601  V101041 2025-11-02    1.0
336602  V101041 2025-11-03    1.0 


 ====== TABLA DETALLADA FPBC1 ====== 

   SERIE      FECHA  FPBC1  C1 H1 %  C1 H2 %  C1 H3 %  C1 H0 %
0  59571 2016-10-23    1.0     0.34      NaN      NaN     0.37
1  59571 2006-01-29    1.0     0.36      NaN      NaN     0.40
2  59571 1996-04-16    NaN      NaN      NaN      NaN      NaN
3  59572 2016-09-18    NaN      NaN      NaN      NaN      NaN
4  59572 2006-01-29    1.0     0.37      NaN      NaN     0.34 


 ====== TABLA DETALLADA EXTENDIDA FPBC1 ====== 

          SERIE      FECHA  FPBC1  C1 H1 %  C